<a href="https://colab.research.google.com/github/RodolfoFerro/satelitesyneuronas/blob/main/notebooks/Segmentaci%C3%B3n_sem%C3%A1ntica_con_UNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentación semántica con UNet


### Trabajos relacionados y avances recientes


Ha habido varios trabajos de investigación y avances recientes que han contribuido al desarrollo de nuevas arquitecturas, técnicas de entrenamiento mejoradas y aplicaciones emergentes.

- **UNet:** Es ampliamente utilizada en el campo de la segmentación de imágenes, pero también se ha aplicado con éxito en tareas de denoising.
- **Variational Autoencoders (VAEs):** Los VAEs son una variante de los autoencoders que se utilizan para el aprendizaje de distribuciones latentes. Han demostrado ser efectivos en el denoising de imágenes al aprender representaciones latentes que siguen una distribución probabilística, lo que permite una generación más controlada y realista de imágenes limpias.
- **GANs y Autoencoders Generativos (GANs-AE):** La combinación de las GANs y los AE ha llevado al desarrollo de los GANs-AE. Estos modelos aprovechan la capacidad de los GANs para generar imágenes realistas y los autoencoders para aprender representaciones latentes eficientes. Los GANs-AE han demostrado ser efectivos en el denoising y la generación de imágenes de alta calidad.


#### **Tareas en el campo de la visión artificial**

1. **Clasificación de imágenes:** La tarea de clasificación de imágenes implica asignar una etiqueta o categoría a una imagen de entrada. Esto implica entrenar un modelo para reconocer y distinguir diferentes objetos, personas o escenas en una imagen.
2. **Detección de objetos:** La detección de objetos implica localizar y clasificar múltiples objetos en una imagen. El objetivo es detectar la presencia y la ubicación de objetos específicos en una escena, a menudo utilizando cuadros delimitadores para delinear las regiones donde se encuentran los objetos.
3. **_Denoising_ o reconstrucción de imágenes:** Consiste en eliminar o reducir el ruido presente en una imagen, obteniendo una versión más limpia y clara. Esta tarea es relevante en áreas como la fotografía, la medicina y la seguridad..
4. **Segmentación semántica:** La segmentación semántica implica asignar una etiqueta a cada píxel de una imagen para identificar y delimitar las diferentes regiones o objetos presentes. El objetivo es comprender la estructura y el contenido de una imagen a nivel de píxel.
5. **Detección de rostros:** La detección de rostros es una tarea específica de la visión artificial que implica detectar y localizar los rostros en una imagen. Es ampliamente utilizado en aplicaciones de reconocimiento facial, análisis de emociones y sistemas de seguridad.
6. **Reconocimiento y verificación facial:** El reconocimiento facial se refiere a la tarea de identificar y reconocer a una persona específica a partir de una imagen o secuencia de imágenes. La verificación facial se enfoca en verificar si una imagen de rostro coincide con una identidad específica.
7. **Estimación de pose:** La estimación de pose se refiere a la tarea de determinar la posición y orientación de un objeto o persona en una imagen. Esto implica detectar y rastrear las articulaciones o puntos clave en una imagen para comprender la postura y el movimiento.
8. **Estimación de profundidad:** La estimación de profundidad implica inferir la información de la distancia o la profundidad de los objetos en una imagen. Es útil en aplicaciones de realidad virtual, conducción autónoma y sistemas de navegación.
9. **Super-resolución:** La super-resolución se refiere a aumentar la resolución o la calidad de una imagen de baja resolución. El objetivo es generar una versión de alta resolución que capture más detalles y claridad.


**Lecturas recomendadas:**

- [Convolution, Padding, Stride, and Pooling in CNN](https://medium.com/analytics-vidhya/convolution-padding-stride-and-pooling-in-cnn-13dc1f3ada26)
- [Why do we need conv2d_transpose?
](https://medium.com/@vaibhavshukla182/why-do-we-need-conv2d-transpose-2534cd2a4d98)
- [Deconvolution](https://vincmazet.github.io/bip/restoration/deconvolution.html)
- [Image Segmentation using deconvolution layer in Tensorflow](https://cv-tricks.com/image-segmentation/transpose-convolution-in-tensorflow/)

### **Clonamos el repositorio**

Comenzaremos clonando el repositorio y asignando a la carpeta como la raíz.

In [ ]:
!git clone https://github.com/RodolfoFerro/satelitesyneuronas.git
%cd satelitesyneuronas
!ls

La estructura del código fuente es como sigue:
- `tools/unet/model.py` - Contiene la implementación del U-Net.
- `tools/data.py` - Contiene funciones de utilería para carga de datos.
- `tools/image.py` - Coniene funciones de utilería para cargar imágenes y mostrar los resultados de las inferencias.
- `main.py` - Contiene una sencilla implementación de este cuaderno en un script de Python para entrenar el modelo.

### **Cargamos los datos**

A continuación procedemos a importar algunas bibliotecas y el código base del modelo.

Haremos uso de alunas funciones que permiten cargar datos que encuentras en el folder `data`.

Comenzaremos importando las funciones de los módulos a utilizar.

In [ ]:
from tools.data import train_generator
from tools.data import test_generator
from tools.data import save_results

Procedemos a crear un diccionario de configuración para cargar datos.

In [ ]:
data_gen_args = dict(
    rotation_range=0.2,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = train_generator(
    2, 'assets/data/membrane/train',
    'image', 'label',
    data_gen_args,
    save_to_dir=None
)


### **Creamos el modelo**

Ahora, procederemos a crear el modelo. Para ello, dos opciones serán previstas.

**OPCIÓN A:** Creamos nuestro propio U-Net con nuestras propias características, basándonos en la propuesta original:

<center>
    <img src="https://production-media.paperswithcode.com/methods/Screen_Shot_2020-07-07_at_9.08.00_PM_rpNArED.png" width="60%">
</center>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam


def unet(pretrained_weights=None, input_size=(256, 256, 1)):
    """U-Net model constructor.

    Parameters
    ----------
    pretrained_weights : str
        Path to pretrained weights.
    input_size : tuple
        Spatial size of the expected input image.
    """

    inputs = Input(input_size)

    # Convolution chain #1
    # conv_1 = ...

    # Continua aquí con tu propia implementación...

**OPCIÓN B:** Creamos una instancia del modelo ya implementado y entrenamos con los datos.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

from tools.unet.model import unet

# Replicable experiments
tf.random.set_seed(42)
np.random.seed(42)

model = unet()

model_checkpoint = ModelCheckpoint(
    'unet_membrane.keras',
    monitor='loss',
    verbose=1,
    save_best_only=True
)

In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=300,
    epochs=5,
    callbacks=[model_checkpoint]
)

In [ ]:
import plotly.express as px
import numpy as np

losses = history.history['loss']
eje_x = np.arange(len(losses))

fig = px.line(
    x=eje_x,
    y=losses,
    title='Historia de entrenamiento',
    labels=dict(x='Épocas', y='Error')
)
fig.show()

**Hasta este punto deberías haber entrenado exitosamente un U-Net con algunas imágenes médicas.**

Una vez entrenado el modelo, podemos realizar pruebas de inferencia con el conjunto de pruebas que se encuentra en la misma carpeta de datos.

In [ ]:
test_gen = test_generator('assets/data/membrane/test')
results = model.predict(test_gen, verbose=True)
save_results('data/results', results)


### **Resultados gráficos**

El código base provee algunas funciones para cargar, inferir y crear máscaras de los resultados al trabajar sobre algunas imágenes.

Procedemos a importar las funciones del módulo de imágenes.

In [ ]:
from tools.image import load_test_image
from tools.image import inference_over_image
from tools.image import create_mask
from tools.image import overlay_mask

Cargamos una imagen del directorio de prueba, especificando con un número entero el índice de alguna de las 30 imágenes (`[0, 29]`).

In [ ]:
img = load_test_image(0)

Usamos el modelo previamente entrenado para inferir sobre la imagen previamente cargada.

In [ ]:
out = inference_over_image(model, img)

Creamos una máscara a partir de la inferencia.

In [ ]:
mask = create_mask(out)

Sobreponemos la máscara en la imágen original para validar el resulatdo.

In [ ]:
res = overlay_mask(img, mask)

**¡Felicidades! Has utilizado exitosamente tu modelo entrenado sobre algunas imágenes médicas.**

**Reto:** Yo me he encargado de enfocarme en utilizar las detecciones para la identificación de pared celular, sin embargo, puedes modificar o crear tus propias funciones para la detección celular completa.

Por otro lado, no debes limitarte a ello, sino que puedes crear o cargar tu propio conjunto de datos para segmentar otro tipo de elementos, como las mitocondrias (echa un vistazo al [Electron Microscopy Dataset](https://www.epfl.ch/labs/cvlab/data/data-em/)).

---
> Contenido curado por **Rodolfo Ferro**. Contacto: [ferro@cimat.mx](ferro@cimat.mx) <br>
[**Clubes de Ciencia México**](https://clubesdeciencia.mx/), 2025.